In [ ]:
import numpy as np
import anndata as ad
import scanpy as sc
from rosa.preprocessing import (
    clean_cells_genes,
)

RAW_ADATA_PT = "/Users/nsofroniew/Documents/data/multiomics/cell_census/tabula_sapiens_by_features.h5ad"
EMBEDS_ADATA_PT = "/Users/nsofroniew/Documents/data/multiomics/cell_census/tabula_sapiens_by_features_with_embeds_new_norm.h5ad"


In [ ]:
adata = ad.read_h5ad(EMBEDS_ADATA_PT)

In [ ]:
adata.layers['counts'].sum(axis=1)

In [ ]:
adata.layers['counts_normalized'] = adata.layers['counts'].copy()
sc.pp.normalize_total(adata, target_sum=1e5, layer='counts_normalized')

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(adata.layers['counts_normalized'].flatten());

In [ ]:
from rosa.preprocessing import bin_expression, reconstruct_expression

In [ ]:
bin_expression(adata, 128)

In [ ]:
reconstruct_expression(adata)

In [ ]:
((adata.X - adata.layers['reconstructed'])**2).mean()

In [ ]:
# Plot residuals
plt.hist((adata.layers['reconstructed'] - adata.X).ravel(), bins=1000);
plt.xlim([-.25, .25]);

In [ ]:
# Identify cells and genes not trained on (when possible)
adata.layers['prediction'] = adata.layers['reconstructed']
test_genes = np.logical_not(adata.var["train"])
test_cells = np.logical_not(adata.obs["train"])
adata_test = adata[test_cells, test_genes]
sc.tl.dendrogram(adata_test, groupby="label", use_rep="X")


In [ ]:
from rosa.plotting import plot_marker_gene_heatmap

In [ ]:
marker_genes_dict = adata_test.obs.set_index('label').to_dict()['marker_feature_name']
plot_marker_gene_heatmap(adata_test, marker_genes_dict)

In [ ]:
plt.hist(adata.layers['binned'].flatten(), bins=25, density=True);

In [ ]:
plt.hist(adata.X.flatten(), bins=250, density=True);
plt.ylim([0, 1]);
plt.xlim([0, 8])

In [ ]:
adata.X

In [ ]:
from typing import Union

In [ ]:
adata.layers['X']

In [ ]:
np.empty((10, 0))[0]

In [ ]:
from enum import Enum, auto

class EmbeddingType(Enum):
    JOINT = auto()
    VAR = auto()
    OBS = auto()

In [ ]:
EmbeddingType.JOINT

In [ ]:
list(EmbeddingType.__members__)

In [ ]:
adata.X = np.ceil(adata.X)
sc.pp.filter_genes(adata, min_cells=1)
sc.experimental.pp.normalize_pearson_residuals(adata)
adata.X[adata.X<0] = 0

In [ ]:
2**10

In [ ]:
import torch

In [ ]:
np.isinf(adata.X).sum()

In [ ]:
np.isnan(adata.X).sum()

In [ ]:
sc.pp.log1p(adata)

In [ ]:
adata.X

In [ ]:
torch.tensor([0])

In [ ]:
adata.X = np.ceil(adata.X)
adata.layers["counts"] = adata.X.copy()

In [ ]:
sc.pp.filter_genes(adata, min_cells=1)

In [ ]:
sc.pp.filter_cells(adata, min_genes=1)

In [ ]:
adata

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(adata.layers['counts'].sum(axis=1));

In [ ]:
print(adata.layers['counts'].sum(axis=1).mean() / 1e5)
print(adata.layers['counts'].sum(axis=1).var() / 1e10)

In [ ]:
adata.layers["counts_normalized_total"] = adata.X.copy()
sc.pp.normalize_total(adata, 1e5, layer="counts_normalized_total")

In [ ]:
print(adata.layers['counts_normalized_total'].sum(axis=1).mean() / 1e5)
print(adata.layers['counts_normalized_total'].sum(axis=1).var() / 1e10)

In [ ]:
adata.layers["counts_normalized_pearson"] = adata.X.copy()
adata.layers['counts_normalized_pearson'] = np.ceil(adata.layers['counts_normalized_pearson'])
sc.experimental.pp.normalize_pearson_residuals(adata, layer="counts_normalized_pearson", theta=1e2)

In [ ]:
np.isnan(adata.layers['counts_normalized_pearson']).sum()

In [ ]:
print(adata.layers['counts_normalized_pearson'].sum(axis=1).mean())
print(adata.layers['counts_normalized_pearson'].sum(axis=1).var())

In [ ]:
plt.hist(adata.layers['counts_normalized_pearson'].flatten(), np.linspace(0, 100, 1000));

In [ ]:
plt.hist(adata.layers['counts'].flatten(), np.linspace(0, 100, 1000));

In [ ]:
np.log1p(0.1)

In [ ]:
(adata.layers['counts_normalized_pearson'] - adata.layers['counts']).max()

In [ ]:
adata.layers['counts_normalized_pearson'] = np.round(adata.layers['counts_normalized_pearson'])

In [ ]:
np.isnan(adata.layers['counts_normalized_pearson'])[0]

In [ ]:
adata.layers['counts_normalized_pearson'].min()

In [ ]:
adata.uns['pearson_residuals_normalization']

In [ ]:
TABULA_SAPIENS_BY_CELL_TYPE_WITH_EMBEDS_PT = "/Users/nsofroniew/Documents/data/multiomics/cell_census/tabula_sapiens_by_features_with_embeds_new_norm.h5ad"

In [ ]:
adata_norm = ad.read_h5ad(TABULA_SAPIENS_BY_CELL_TYPE_WITH_EMBEDS_PT)

In [ ]:
y = np.exp(adata_norm.X) - 1

In [ ]:
y

In [ ]:
import scanpy as sc

In [ ]:
adata_norm.X = adata_norm.layers['counts'].copy()

In [ ]:
sc.pp.normalize_total(adata_norm, 1e5)

In [ ]:
(abs(adata_norm.X - y)).max()

In [ ]:
# from scipy.special import kl_div

# y_hat = np.asarray(adata[keep_cells].X.flatten())
# y = np.asarray(adata[keep_cells].layers['prediction'].flatten())

# kl_div(y, y_hat).mean()

In [ ]:
from scipy.stats import kstest, poisson

y_hat = np.asarray(adata[keep_cells].X.flatten())
y = np.asarray(adata[keep_cells].layers['prediction'].flatten())

kstest(y_hat, y)

In [ ]:
kstest(y, 'poisson', args=(np.mean(y),))

In [ ]:
from scipy.stats import kstest, poisson

poisson_dist = poisson(np.mean(y))

In [ ]:
y_p = poisson_dist.rvs(size=10000)

In [ ]:
hist, _ = np.histogram(y, bins=bins)
hist_hat, _ = np.histogram(y_hat, bins=bins)
plt.bar(bins[:-1], hist/hist.sum(), width=10/1000)
plt.bar(bins[:-1], hist_hat/hist_hat.sum(), alpha=0.5, width=10/1000)
plt.ylim([0, 1e-2]);
plt.xlim([0, 8]);

In [ ]:
poisson_dist = poisson(np.mean(y))
y_new = poisson_dist.rvs(size=10000)


hist, _ = np.histogram(y, bins=bins)
hist_new, _ = np.histogram(y_new, bins=bins)
plt.bar(bins[:-1], hist/hist.sum(), width=10/1000)
plt.bar(bins[:-1], hist_new//hist_new.sum(), alpha=0.5, width=10/1000)
plt.ylim([0, 1e-2]);
plt.xlim([0, 8]);

In [ ]:
initial_params = [.2, 0.1, 1.2, 0.0.001]

result = minimize(negative_binomial, initial_params, args=(y,), method='Nelder-Mead')

In [ ]:
# Extract the optimal parameters
r1, p1, r2, p2 = result.x
data = y

In [ ]:
from scipy.stats import nbinom

r1, p1, r2, p2 = (0.1, 0.1, 1.1, 0.1)

# nbinom_dist_1 = nbinom(9.1, 0.6)
# nbinom_dist_2 = nbinom(1.2, .001)
y_new = (nbinom.rvs(.2, 0.1, size=10000) + nbinom.rvs(1.2, 0.001, size=10000)) / 1000


hist, _ = np.histogram(y, bins=bins)
hist_new, _ = np.histogram(y_new, bins=bins)
plt.bar(bins[:-1], hist/hist.sum(), width=10/1000)
plt.bar(bins[:-1], hist_new/hist_new.sum(), alpha=0.5, width=10/1000)
plt.ylim([0, 1e-2]);
plt.xlim([0, 8]);

In [ ]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import nbinom
import matplotlib.pyplot as plt

# Generate sample data
data = y

# Define the negative binomial function
def negative_binomial(params, data):
    r1, p1, r2, p2 = params
    pmf1 = nbinom.pmf(1000 * data, r1, p1)
    pmf2 = nbinom.pmf(1000 * data, r2, p2)
    return -np.log(pmf1 + pmf2).sum()

# Define the initial values for the parameters
initial_params = (0.1, 0.1, 1.1, 0.1)


# Minimize the negative binomial function using the Nelder-Mead algorithm
result = minimize(negative_binomial, initial_params, args=(data,), method='Nelder-Mead')

# Extract the optimal parameters
r1, p1, r2, p2 = result.x

# Plot the histogram of the data
plt.hist(data, bins=30, density=True, alpha=0.5, label='Data')

# Plot the sum of the negative binomials
x = np.arange(0, data.max())
pmf1 = nbinom.pmf(x, r1, p1)
pmf2 = nbinom.pmf(x, r2, p2)
plt.plot(x, pmf1 + pmf2, 'r-', lw=2, label='Sum of Negative Binomials')

plt.xlabel('x')
plt.ylabel('Probability')
plt.legend()
plt.show()


In [ ]:
try:
  print(x)
except NameError:
  print("Variable x is not defined")

In [ ]:
import torch

In [ ]:
torch.long

In [ ]:
a = torch.tensor(2.3).type(torch.long)

In [ ]:
adata.varm['embedding'].shape

In [ ]:
import matplotlib.pyplot as plt

plt.hist(adata.varm['embedding'].ravel(), bins=2000);
plt.xlim([-0.5, 0.5]);

In [ ]:
adata.varm['embedding']

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# fit pca on training data
pca = PCA()
pca.fit(adata.varm['embedding'])


In [ ]:
E = pca.transform(adata.varm['embedding'])

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_));
plt.xlim([0, 3042])

In [ ]:
np.cumsum(pca.explained_variance_ratio_)[512]

In [ ]:
plt.hist(E[:, :512].ravel(), bins=2000);
plt.xlim([-1.5, 1.5]);

In [ ]:
E.shape

In [1]:
import anndata as ad
from rosa.preprocessing import (
    calculate_gene_embeddings_pca,
)


EMBEDS_ADATA_PT = "/Users/nsofroniew/Documents/data/multiomics/cell_census/tabula_sapiens_by_features_with_embeds_new_norm.h5ad"

Global seed set to 0
/Users/nsofroniew/opt/anaconda3/envs/multiomics/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/Users/nsofroniew/opt/anaconda3/envs/multiomics/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [2]:
adata = ad.read_h5ad(EMBEDS_ADATA_PT)
adata = calculate_gene_embeddings_pca(adata, 256)

In [3]:
adata.uns["embedding_pca"]

{'explained_variance': 0.918266625236611}

In [4]:
adata.write_h5ad(EMBEDS_ADATA_PT)

In [ ]:
adata.varm['embedding_pca'].shape

In [ ]:
adata.varm

In [10]:
ADATA_BULK_PT = "/Users/nsofroniew/Documents/data/multiomics/cell_census/tabula_sapiens_pbulk.h5ad"
adata = ad.read_h5ad(ADATA_BULK_PT)


In [16]:
adata.var.set_index("feature_id")

,soma_joinid,feature_name,feature_length
feature_id,,,
ENSG00000121410,0,A1BG,3999
ENSG00000268895,1,A1BG-AS1,3374
ENSG00000118017,10,A4GNT,1779
ENSG00000129968,100,ABHD17A,6163
ENSG00000236469,1000,AC007040.8,570
...,...,...,...
ENSG00000256789,9995,RP11-697H9.2,423
ENSG00000255015,9996,RP11-716H6.1,507
ENSG00000255219,9997,RP11-716H6.2,571


In [6]:
base_pt = '/home/ec2-user/enformer/Homo_sapiens.GRCh38.genes.enformer_embeddings'
var_id = 'ENSG00000280445'
full_pt = f'{base_pt}/{var_id}.pt'

In [3]:
import torch

In [71]:
for i in range(64):
    var = torch.load(full_pt, map_location='cpu')['embedding']
    var = torch.from_numpy(var).type(torch.float32)

In [21]:
type(var)

torch.Tensor

In [11]:
896 // 2

448

In [101]:
a = var.unsqueeze(0).unsqueeze(-3)
a.shape

torch.Size([1, 1, 896, 3072])

In [95]:
fc = torch.nn.Conv2d(1, 10, (896, 1))

In [103]:
fc(a).view(a.shape[0], -1).shape

torch.Size([1, 30720])

In [ ]:
view(a.shape[0], -1)

In [218]:
import torch.nn.functional as F


class ToTensor(torch.nn.Module):
    """Convert ``numpy.ndarray`` to tensor.
    """
    def __init__(self, dtype: torch.dtype = torch.float32) -> None:
        super().__init__()
        self.dtype = dtype

    def forward(self, tensor: np.ndarray) -> torch.Tensor:
        return torch.from_numpy(tensor).type(torch.float32)

class CountNormalize(torch.nn.Module):
    """Normalize a tensor to a fixed total counts.
    """
    def __init__(self, total_counts=1):
        super().__init__()
        self.total_counts = total_counts

    def forward(self, tensor: torch.Tensor) -> torch.Tensor:
        return self.total_counts * F.normalize(tensor, p=1.0, eps=1e-12)


class Log1p(torch.nn.Module):
    """Log1p normalize a tensor.
    """
    def __init__(self):
        super().__init__()

    def forward(self, tensor: torch.Tensor) -> torch.Tensor:
        return torch.log1p(tensor)


class QuantileNormalize(torch.nn.Module):
    """Normalize a tensor by quantiles.
    """
    def __init__(self, n_bins):
        super().__init__()
        self.n_bins = n_bins

    def forward(self, tensor: torch.Tensor) -> torch.Tensor:
        boundaries = torch.quantile(tensor, torch.linspace(0, 1, self.n_bins))
        return torch.bucketize(tensor, boundaries)

In [219]:
class ExpressionTransform(torch.nn.Sequential):
    def __init__(self, cfg):
        # Add base transform
        transforms = [ToTensor()]

        if cfg.total_counts is not None:
            transforms.append(CountNormalize(cfg.total_counts))

        if cfg.log1p:
            transforms.append(Log1p())

        if cfg.n_bins is not None:
            transforms.append(QuantileNormalize(cfg.n_bins))

        super().__init__(*transforms)


from dataclasses import dataclass
from typing import Optional


@dataclass
class ExpressionTransformConfig:
    total_counts: Optional[int] = None
    log1p: Optional[bool] = None
    n_bins: Optional[int] = None

In [220]:
exp_cfg = ExpressionTransformConfig(n_bins=10)

In [221]:
tf = ExpressionTransform(exp_cfg)

In [222]:
torch.quantile(ToTensor()(X).unsqueeze(0), torch.linspace(0, 1, 5), dim=-1, keepdim=True).shape

torch.Size([5, 1, 3, 1])

In [223]:
ToTensor()(X).unsqueeze(0).shape

torch.Size([1, 3, 30])

In [224]:
tf(np.random.rand(20))

tensor([3, 4, 5, 1, 5, 9, 7, 2, 0, 1, 2, 7, 6, 9, 4, 9, 6, 3, 8, 8])

In [ ]:
X[1]

In [169]:
a = torch.nn.Sequential(ToTensor(), CountNormalize(10), Log1p(), QuantileNormalize(10))
b = torch.nn.Sequential(ToTensor(), QuantileNormalize(10))

In [200]:
X = np.random.rand(3, 30)
X [0, :3] = 0
X [0, :] = X [0, :] * 1000
# X = torch.randint(1, 5, size=(3, 3))

In [201]:
X

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 9.04933046e+02,
        5.86185246e+02, 3.67209684e+02, 9.28032463e+02, 9.43061788e+02,
        6.61347813e+02, 9.09201383e+02, 6.52564336e+01, 7.99837338e+02,
        2.61558041e+02, 7.52127084e+02, 9.60963693e+02, 3.92288179e+02,
        8.56678094e+02, 6.97483553e+02, 1.11283442e+02, 7.34695066e+02,
        8.55954620e+02, 8.99893485e+02, 2.86306631e+02, 1.61621586e+02,
        3.46326170e+02, 6.44120733e+02, 9.49631103e+02, 4.06631873e+00,
        9.37364444e+02, 9.67401543e+02],
       [2.60777832e-01, 9.44569420e-01, 9.31749530e-01, 1.39988490e-01,
        5.88439432e-01, 8.94237201e-01, 2.34748228e-01, 1.43530256e-01,
        5.14576896e-01, 1.99584205e-01, 5.56390586e-01, 3.41238556e-01,
        6.24677666e-01, 4.59784231e-01, 5.80895454e-01, 4.41532664e-01,
        5.67134862e-02, 1.51184503e-01, 7.05252488e-01, 6.01157209e-01,
        9.84628333e-01, 4.45089557e-01, 2.13056719e-01, 4.27250153e-01,
        6.44899373e-01,

In [167]:
a(X)

tensor([[0, 0, 0, 4, 3, 6, 7, 8, 6, 3, 5, 1, 5, 9, 8, 7, 2, 9, 2, 2, 6, 5, 4, 7,
         8, 5, 4, 2, 9, 9],
        [7, 6, 2, 1, 5, 1, 4, 4, 6, 5, 8, 6, 7, 9, 9, 3, 3, 5, 1, 1, 5, 8, 3, 9,
         9, 3, 3, 2, 6, 9],
        [7, 2, 1, 4, 8, 4, 3, 8, 2, 7, 2, 6, 9, 7, 8, 7, 5, 6, 4, 5, 8, 3, 8, 6,
         1, 2, 7, 3, 4, 4]])

In [170]:
b(X)

tensor([[0, 0, 0, 4, 3, 6, 6, 8, 5, 3, 5, 1, 5, 8, 7, 7, 2, 9, 2, 2, 6, 5, 4, 7,
         8, 5, 4, 2, 8, 9],
        [7, 6, 2, 1, 5, 1, 4, 4, 6, 6, 8, 6, 8, 9, 9, 3, 3, 5, 1, 1, 5, 8, 3, 9,
         9, 3, 3, 2, 6, 9],
        [7, 2, 1, 4, 9, 4, 3, 8, 2, 7, 2, 6, 9, 7, 9, 7, 5, 7, 4, 5, 8, 3, 8, 6,
         1, 2, 7, 3, 4, 4]])

In [168]:
a(X).sum(dim=1)

tensor([146, 150, 151])

In [77]:
20 * 512 / 1e3

10.24

In [143]:
type(torch.float32)

torch.dtype

In [26]:
import numpy as np

In [31]:
np.unravel_index(10, (9, 2))

(5, 0)

In [34]:
EMBEDS_ADATA_PT = '/Users/nsofroniew/Documents/data/multiomics/cell_census/tabula_sapiens_by_features_with_embeds_new_norm.h5ad'

In [50]:
EMBEDS_ADATA_PT_2 = "/Users/nsofroniew/Documents/data/multiomics/cell_census/tabula_sapiens_by_features_raw.h5ad"


In [35]:
import anndata as ad

In [51]:
adata = ad.read_h5ad(EMBEDS_ADATA_PT)
adata_2 = ad.read_h5ad(EMBEDS_ADATA_PT_2)

In [64]:
adata.layers['counts'][0, 0]

0.032808773

In [62]:
adata_2.X.sum(axis=1).mean()

91241.07

In [45]:
adata.X[0, 0]

0.10028871

In [49]:
adata.layers['binned'][0, 0]

44

In [55]:
adata_2.X[0, 0]

0.032808773

In [79]:
from enum import Enum, auto
from typing import Optional


class ExpressionActivations(Enum):
    SOFTPLUS = auto()
    SOFTMAX = auto()

In [73]:
str(ExpressionActivations.SOFTPLUS)

'ExpressionActivations.SOFTPLUS'

In [80]:
ExpressionActivations.SOFTPLUS.name.lower()

'softplus'

In [81]:
import torch.nn as nn

In [82]:
a=nn.Sequential()

In [84]:
a.append(nn.Linear(1, 1))

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=True)
  (1): Linear(in_features=1, out_features=1, bias=True)
)

In [85]:
body = nn.Identity()

In [87]:
body(torch.rand(10, 10)).shape

torch.Size([10, 10])

In [89]:
map((body, body), (torch.rand(10, 10), torch.rand(10, 10)))

In [102]:
torch.add((torch.rand(10, 10), torch.rand(10, 10))).shape

TypeError: add() received an invalid combination of arguments - got (tuple), but expected (Tensor input, Tensor other, *, Number alpha, Tensor out)

In [101]:
torch.cat((torch.rand(10, 10), torch.rand(10, 10)), dim=-1).shape

torch.Size([10, 20])

In [127]:
from typing import Tuple

class AttentionEmbeds(nn.Module):
    def __init__(self, in_dim: Tuple[int, int], out_dim) -> None:
        super(AttentionEmbeds, self).__init__()
        
        self.value = nn.Parameter(torch.randn(out_dim))
        self.activation = nn.GELU()
        self.out_dim = out_dim

    def forward(self, x: Tuple[torch.Tensor, torch.Tensor]) -> torch.Tensor:
        atten = self.activation(torch.einsum('...i, ...i ->...', *x))
        return torch.einsum('..., i -> ...i', atten, self.value)

In [110]:
x_1 = torch.rand((10, 20))
x_2 = torch.rand((10, 20))
v = torch.rand(20)

In [109]:
atten = torch.einsum('...i, ...i ->...', x_1, x_2)

In [115]:
out = torch.einsum('..., i -> ...i', atten, v)

In [126]:
nn.Parameter(torch.randn(20))

Parameter containing:
tensor([ 0.9391, -2.6665,  1.7923,  0.1971,  0.0085, -0.5108,  1.6944, -0.3136,
        -0.0164, -0.9281,  1.5383,  0.5811,  0.5446, -0.3123, -0.4335, -0.1784,
         1.1701, -0.0980,  0.4092,  1.2190], requires_grad=True)

In [128]:
a = AttentionEmbeds((20, 20), 30)

In [131]:
a((x_1, x_2)).shape

torch.Size([10, 30])

In [156]:
X = np.random.rand(100, 896, 3072)

In [137]:
PT = '/Users/nsofroniew/Documents/data/multiomics/enformer/scratch'

In [168]:
import zarr

In [169]:
z1 = zarr.open(PT + '/example.zarr', mode='w', shape=(1000, 896, 3072), chunks=(1, None, None), dtype='float32')

In [170]:
for i in range(10):
    z1[i * 100: (i+1)*100] = X

In [171]:
z2 = zarr.open(PT + '/example.zarr', mode='r')

In [144]:
from time import time

In [207]:
start = time()
z2[893]
stop = time()
print(stop - start)

0.004496097564697266


In [184]:
torch.save({'results': X[0]}, PT + '/example_0.pt')

In [201]:
start = time()
torch.load(PT + '/example_0.pt')
stop = time()
print(stop - start)

0.09557294845581055


In [215]:
class ZarrDataset(torch.utils.data.Dataset):
    def __init__(self, path: str):
        super(ZarrDataset, self).__init__()

        self.path = path

        # self.array = zarr.open(path, mode='r')

    def __len__(self) -> int:
        return self.array.shape[0]

    def __getitem__(self, idx):
        return 3 #self.array[idx]

In [218]:
ds = ZarrDataset(PT + '/example.zarr')
dl = torch.utils.data.DataLoader(ds, shuffle=False, num_workers=2)

In [219]:
start = time()
for batch in iter(dl):
    pass
stop = time()
print(stop - start)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/nsofroniew/opt/anaconda3/envs/multiomics/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
  File "/Users/nsofroniew/opt/anaconda3/envs/multiomics/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
    exitcode = _main(fd, parent_sentinel)  File "/Users/nsofroniew/opt/anaconda3/envs/multiomics/lib/python3.9/multiprocessing/spawn.py", line 126, in _main

  File "/Users/nsofroniew/opt/anaconda3/envs/multiomics/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
    AttributeErrorself = reduction.pickle.load(from_parent)
: Can't get attribute 'ZarrDataset' on <module '__main__' (built-in)>
AttributeError: Can't get attribute 'ZarrDataset' on <module '__main__' (built-in)>


RuntimeError: DataLoader worker (pid(s) 29075) exited unexpectedly